In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
import random
import decimal

Script to search racing post and get basic information about pedigree, stallion performance when racing

Then get detailed progeny statistics

In [2]:
df1_cols = ['age',
           'sire',
           'dam',
           'dam_sire',
           'stud_fee',
           'total_starts',
           'total_earnings',
           'top_rpr',
           'sire_comments'
           ]

In [3]:
cols_df = []

info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']

for i in info_to_take:
    new1 = str(i) + "_win_runners"
    new2 = str(i) + "_wins"
    new3 = str(i) + "_runs"
    new4 = str(i) + "_earnings"
    
    cols_df.append(new1)
    cols_df.append(new2)
    cols_df.append(new3)
    cols_df.append(new4)

cols_df

['Flat_win_runners',
 'Flat_wins',
 'Flat_runs',
 'Flat_earnings',
 'Turf_win_runners',
 'Turf_wins',
 'Turf_runs',
 'Turf_earnings',
 'All-weather_win_runners',
 'All-weather_wins',
 'All-weather_runs',
 'All-weather_earnings',
 'Jumps_win_runners',
 'Jumps_wins',
 'Jumps_runs',
 'Jumps_earnings',
 'Chase_win_runners',
 'Chase_wins',
 'Chase_runs',
 'Chase_earnings',
 'Hurdles_win_runners',
 'Hurdles_wins',
 'Hurdles_runs',
 'Hurdles_earnings',
 'NHF_win_runners',
 'NHF_wins',
 'NHF_runs',
 'NHF_earnings',
 'Broodmare sires_win_runners',
 'Broodmare sires_wins',
 'Broodmare sires_runs',
 'Broodmare sires_earnings',
 '2yo_win_runners',
 '2yo_wins',
 '2yo_runs',
 '2yo_earnings',
 'First Crop_win_runners',
 'First Crop_wins',
 'First Crop_runs',
 'First Crop_earnings',
 '5-6f_win_runners',
 '5-6f_wins',
 '5-6f_runs',
 '5-6f_earnings',
 '7-9f_win_runners',
 '7-9f_wins',
 '7-9f_runs',
 '7-9f_earnings',
 '10-11f_win_runners',
 '10-11f_wins',
 '10-11f_runs',
 '10-11f_earnings',
 '12-13f_win_

In [4]:
horse_list = pd.read_excel('../stallion_list.xlsx')
horse_list.info()


horse_ls = list(horse_list['STALLION NAME'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STALLION NAME  311 non-null    object 
 1   STANDING       310 non-null    object 
 2   CTRY           310 non-null    object 
 3   STUD FEE       310 non-null    object 
 4   SIRE           310 non-null    object 
 5   SIRE LINE      310 non-null    object 
 6   YEAR TO STUD   310 non-null    float64
dtypes: float64(1), object(6)
memory usage: 17.5+ KB


In [5]:
def find_the_correct_google_link(horse):
    url2='http://www.google.com'
    browser.get(url2)
        
    time.sleep(2)
    
    search = browser.find_element_by_name('q')
    search.send_keys(str(horse) + ' racing post progeny')
    search.send_keys(Keys.RETURN) # hit return after you enter search text
    
    time.sleep(2)
    
    try:
        search_form=browser.find_elements_by_xpath('//h3[@class="LC20lb DKV0Md"]')
        time.sleep(2)
        search_form[0].click()
    except:
        search_form=browser.find_elements_by_xpath('//h3[@class="LC20lb DKV0Md"]')
        time.sleep(2)
        search_form[4].click()
        
# browser.find_element(By.XPATH, '(//h3[@class="LC20lb"])[1]/a').click()

In [6]:
def weatherby(horse):
        
    search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down hidden-sm-down"]')
    search_form[0].click()
        
    time.sleep(3)
        
    timeout = 10
    WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='hp-raceRecords__label']")))
        
    element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]
    year_perf=titles[-36:]
    year_perf_2 = titles[-18:]
    top_rpr = titles[10]
    total_earnings = titles[7]
    total_starts=titles[1]

    age_element = browser.find_elements_by_xpath("//dt[@class='pp-definition__term']")
    age_element_info = [x.text for x in age_element]
    age=age_element_info[0]

    sire = browser.find_elements_by_xpath("//a[@class='ui-link ui-link_table js-popupLink hp-horseDefinition__link']")
    sire_info = [x.text for x in sire]

    stud = browser.find_elements_by_xpath("//span[@class='hp-nameRow__studFee__value']")
    stud_fee = [x.text for x in stud]
    
    comments = browser.find_elements_by_xpath("//dd[@class='pp-definition__description']")
    sire_comments = [x.text for x in comments]

    table_option1 = [age,
                    sire_info[0],
                    sire_info[1],
                    sire_info[2],
                    stud_fee[0],
                    total_starts,
                    total_earnings,
                    top_rpr,
                    sire_comments[5]]

    df1.loc[horse,:]=table_option1
    
    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab"]')
    search_form[1].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements_by_xpath('//a[@class="ui-link ui-link_primary ui-popoverList__link pp-subTabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))
    
    element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]


    info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                    'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                    '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']

    time.sleep(float(decimal.Decimal(random.randrange(150, 300))/100))
    
    for i in info_to_take:
        if i in titles:
            col1 = str(i) + "_win_runners"
            col2 = str(i) + "_wins"
            col3 = str(i) + "_runs"
            col4 = str(i) + "_earnings"

            find_index = titles.index(i)

            win_runners = titles[find_index + 1]
            wins = titles[find_index + 3]
            runs = titles[find_index + 4]
            earnings = titles[find_index + 8]

            df2.loc[horse,[col1, col2, col3, col4]]=[win_runners, wins, runs, earnings]




    return df1, df2


In [7]:
browser = webdriver.Chrome()
browser.get('http://www.racingpost.com')
    
df1=pd.DataFrame(columns=df1_cols)
df2=pd.DataFrame(columns=cols_df)

time.sleep(3)

find_the_correct_google_link('Aeroplane')

time.sleep(3)

search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down hidden-sm-down"]')
search_form[0].click()

time.sleep(3)

timeout = 10
WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='hp-raceRecords__label']")))

element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
titles = [x.text for x in element2]
year_perf=titles[-36:]
year_perf_2 = titles[-18:]
top_rpr = titles[10]
total_earnings = titles[7]
total_starts=titles[1]

age_element = browser.find_elements_by_xpath("//dt[@class='pp-definition__term']")
age_element_info = [x.text for x in age_element]
age=age_element_info[0]

sire = browser.find_elements_by_xpath("//a[@class='ui-link ui-link_table js-popupLink hp-horseDefinition__link']")
sire_info = [x.text for x in sire]

stud = browser.find_elements_by_xpath("//span[@class='hp-nameRow__studFee__value']")
stud_fee = [x.text for x in stud]

comments = browser.find_elements_by_xpath("//dd[@class='pp-definition__description']")
sire_comments = [x.text for x in comments]

table_option1 = [age,
                sire_info[0],
                sire_info[1],
                sire_info[2],
                stud_fee[0],
                total_starts,
                total_earnings,
                top_rpr,
                sire_comments[5]]

# df1.loc[horse,:]=table_option1

time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab"]')
search_form[1].click()

time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

search_form=browser.find_elements_by_xpath('//a[@class="ui-link ui-link_primary ui-popoverList__link pp-subTabs__tab"]')
search_form[0].click()

# timeout = 5
# driver.navigate().refresh()
# WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//td[@class='ui-table__cell']")))
time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
titles = [x.text for x in element2]


info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']

time.sleep(float(decimal.Decimal(random.randrange(150, 300))/100))

for i in info_to_take:
    if i in titles:
        col1 = str(i) + "_win_runners"
        col2 = str(i) + "_wins"
        col3 = str(i) + "_runs"
        col4 = str(i) + "_earnings"

        find_index = titles.index(i)

        win_runners = titles[find_index + 1]
        wins = titles[find_index + 3]
        runs = titles[find_index + 4]
        earnings = titles[find_index + 8]

        df2.loc[horse,[col1, col2, col3, col4]]=[win_runners, wins, runs, earnings]


TimeoutException: Message: 


In [8]:
other_click = browser.find_elements_by_xpath('//button[@class="ui-reset-for-btn ui-modal__closeBtn"]')
other_click[0].click()

time.sleep(2)

search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down"]')
search_form[0].click()

timeout = 10
WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//h2[@class='hp-formTable__title']")))


element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
titles = [x.text for x in element2]
top_rpr = titles[10]
total_earnings = titles[7]
total_starts=titles[1]

age_element = browser.find_elements_by_xpath("//dt[@class='pp-definition__term']")
age_element_info = [x.text for x in age_element]
age=age_element_info[0]

sire = browser.find_elements_by_xpath("//a[@class='ui-link ui-link_table js-popupLink hp-horseDefinition__link']")
sire_info = [x.text for x in sire]

stud = browser.find_elements_by_xpath("//span[@class='hp-nameRow__studFee__value']")
if not stud:
    stud_fee = 'not there'
else:
    stud_fee = [x.text for x in stud]

comments = browser.find_elements_by_xpath("//dd[@class='pp-definition__description']")
sire_comments = [x.text for x in comments]

table_option = [age,
                sire_info[0],
                sire_info[1],
                sire_info[2],
                stud_fee[0],
                total_starts,
                total_earnings,
                top_rpr,
                sire_comments[5]]


time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab"]')
search_form[0].click()

time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

search_form=browser.find_elements_by_xpath('//a[@class="ui-link ui-link_primary ui-popoverList__link pp-subTabs__tab"]')
search_form[0].click()

time.sleep(float(decimal.Decimal(random.randrange(100, 150))/100))

element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
titles = [x.text for x in element2]

time.sleep(float(decimal.Decimal(random.randrange(150, 300))/100))

info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']

IndexError: list index out of range

In [12]:
browser.find_elements_by_xpath('//a[@class="ui-popoverList__link pp-subTabs__tab"]')

[]

In [7]:
def non_weatherby(horse):
    other_click = browser.find_elements_by_xpath('//button[@class="ui-reset-for-btn ui-modal__closeBtn"]')
    other_click[0].click()

    time.sleep(2)

    search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down"]')
    search_form[0].click()

    timeout = 10
    WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//h2[@class='hp-formTable__title']")))


    element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]
    top_rpr = titles[10]
    total_earnings = titles[7]
    total_starts=titles[1]

    age_element = browser.find_elements_by_xpath("//dt[@class='pp-definition__term']")
    age_element_info = [x.text for x in age_element]
    age=age_element_info[0]

    sire = browser.find_elements_by_xpath("//a[@class='ui-link ui-link_table js-popupLink hp-horseDefinition__link']")
    sire_info = [x.text for x in sire]

    stud = browser.find_elements_by_xpath("//span[@class='hp-nameRow__studFee__value']")
    if not stud:
        stud_fee = 'not there'
    else:
        stud_fee = [x.text for x in stud]

    comments = browser.find_elements_by_xpath("//dd[@class='pp-definition__description']")
    sire_comments = [x.text for x in comments]

    table_option = [age,
                    sire_info[0],
                    sire_info[1],
                    sire_info[2],
                    stud_fee[0],
                    total_starts,
                    total_earnings,
                    top_rpr,
                    sire_comments[5]]


    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements_by_xpath('//a[@class="ui-link ui-link_primary ui-popoverList__link pp-subTabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(100, 150))/100))

    element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]

    time.sleep(float(decimal.Decimal(random.randrange(150, 300))/100))

    info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                    'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                    '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']


    for i in info_to_take:
        if i in titles:
            col1 = str(i) + "_win_runners"
            col2 = str(i) + "_wins"
            col3 = str(i) + "_runs"
            col4 = str(i) + "_earnings"

            find_index = titles.index(i)

            win_runners = titles[find_index + 1]
            wins = titles[find_index + 3]
            runs = titles[find_index + 4]
            earnings = titles[find_index + 8]

            df2.loc[horse,[col1, col2, col3, col4]]=[win_runners, wins, runs, earnings]




    return df1, df2

In [8]:
def racingpost(horse):
    
    time.sleep(3)
    
    find_the_correct_google_link(horse)
    
    time.sleep(3)
    
    timeout = 10
    try:
        weatherby(horse)
    except:
        non_weatherby(horse)
        
    
#    try:
#        weatherby(horse)
#    except:
#        print(str(horse) + ' probably a non_weatherby horse')

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('http://www.racingpost.com')



[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/jackpickard/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [8]:
search_form=browser.find_elements_by_xpath('//a[@class="CybotCookiebotDialogBodyButton"]')
time.sleep(1)
search_form[1].click()

In [6]:
search_form

[<selenium.webdriver.remote.webelement.WebElement (session="c89d95ec6ffe7bb112b09ea394193702", element="a7ee722f-89a6-4993-bf81-e72bd3a20f76")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c89d95ec6ffe7bb112b09ea394193702", element="80b5a7e5-b929-40ae-936c-9a59c3f3e7a9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c89d95ec6ffe7bb112b09ea394193702", element="ca59ce4b-cc62-47a8-b485-d75c0d18c684")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c89d95ec6ffe7bb112b09ea394193702", element="f5a99300-9ef1-4a05-a889-f9a054095c87")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c89d95ec6ffe7bb112b09ea394193702", element="ae3f519c-267d-449a-9ed1-d038cb5ea4a5")>]

In [9]:
failed = []

browser = webdriver.Chrome()
browser.get('http://www.racingpost.com')
    
df1=pd.DataFrame(columns=df1_cols)
df2=pd.DataFrame(columns=cols_df)

i=0

for horse in horse_ls:
    try:
        racingpost(horse)
    except:
        failed.append(horse)
    i=i+1
    print('{}/{}, failed={}'.format(i, len(horse_ls), len(failed)))


1/317, failed=0
2/317, failed=0
3/317, failed=0
4/317, failed=1
5/317, failed=1
6/317, failed=1
7/317, failed=1
8/317, failed=1
9/317, failed=1
10/317, failed=1
11/317, failed=1
12/317, failed=1
13/317, failed=2
14/317, failed=2
15/317, failed=2
16/317, failed=2
17/317, failed=2
18/317, failed=2
19/317, failed=2
20/317, failed=2
21/317, failed=2
22/317, failed=2
23/317, failed=2
24/317, failed=2
25/317, failed=2
26/317, failed=2
27/317, failed=2
28/317, failed=2
29/317, failed=2
30/317, failed=2
31/317, failed=2
32/317, failed=3
33/317, failed=3
34/317, failed=3
35/317, failed=3
36/317, failed=3
37/317, failed=3
38/317, failed=3
39/317, failed=3
40/317, failed=3
41/317, failed=3
42/317, failed=3
43/317, failed=3
44/317, failed=3
45/317, failed=3
46/317, failed=3
47/317, failed=3
48/317, failed=3
49/317, failed=3
50/317, failed=3
51/317, failed=3
52/317, failed=3
53/317, failed=3
54/317, failed=3
55/317, failed=3
56/317, failed=3
57/317, failed=3
58/317, failed=3
59/317, failed=3
60/317

In [15]:
horse_ls[17]

'Footstepsinthesand'

In [12]:
len(horse_ls)

299

In [10]:
df1.shape

(163, 9)

In [11]:
df2.shape

(104, 84)

In [12]:
df1.to_csv('df1_11may20_stallions.csv')

In [13]:
df2.to_csv('df2_11may20_stallions.csv')

In [14]:
df3 = df2.merge(df1, left_index=True, right_index=True)

In [15]:
df3.to_csv('progeny_stats_11may20_original.csv')

In [20]:
df3.sample().transpose()

,Camelot
Flat_win_runners,103/179
Flat_wins,163
Flat_runs,1273
Flat_earnings,"£3,859,378"
Turf_win_runners,81/163
...,...
stud_fee,"€40,000"
total_starts,10
total_earnings,"£1,926,569"
top_rpr,126


In [ ]:
def non_weatherby_t(horse):
    time.sleep(2)

    search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab hidden-xs-down"]')
    search_form[0].click()

    timeout = 10
    WebDriverWait(browser, timeout).until(EC.visibility_of_element_located((By.XPATH, "//h2[@class='hp-formTable__title']")))


    element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]
    top_rpr = titles[10]
    total_earnings = titles[7]
    total_starts=titles[1]

    age_element = browser.find_elements_by_xpath("//dt[@class='pp-definition__term']")
    age_element_info = [x.text for x in age_element]
    age=age_element_info[0]

    sire = browser.find_elements_by_xpath("//a[@class='ui-link js-popupLink hp-horseDefinition__link']")
    sire_info = [x.text for x in sire]

    stud = browser.find_elements_by_xpath("//span[@class='hp-nameRow__studFee__value']")
    stud_fee = [x.text for x in stud]

    table_option = [age,
                    sire_info[0],
                    sire_info[1],
                    sire_info[2],
                    stud_fee[0],
                    total_starts,
                    total_earnings,
                    top_rpr]


    df1.loc[horse,:]=table_option

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))
    
    search_form=browser.find_elements_by_xpath('//a[@class="ui-tabs__tab pp-tabs__tab"]')
    search_form[0].click()

    time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))

    search_form=browser.find_elements_by_xpath('//a[@class="ui-popoverList__link pp-subTabs__tab"]')
    search_form[1].click()
    
    time.sleep(float(decimal.Decimal(random.randrange(100, 150))/100))

    element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
    titles = [x.text for x in element2]


    info_to_take = ['Flat','Turf','All-weather','Jumps','Chase','Hurdles',
                    'NHF','Broodmare sires','2yo','First Crop','5-6f','7-9f','10-11f',
                    '12-13f','14f+','Heavy','Soft','Gd-sft','Good','Gd-fm','Firm']


    for i in info_to_take:
        if i in titles:
            col1 = str(i) + "_win_runners"
            col2 = str(i) + "_wins"
            col3 = str(i) + "_runs"
            col4 = str(i) + "_earnings"

            find_index = titles.index(i)

            win_runners = titles[find_index + 1]
            wins = titles[find_index + 3]
            runs = titles[find_index + 4]
            earnings = titles[find_index + 8]

            df2.loc[horse,[col1, col2, col3, col4]]=[win_runners, wins, runs, earnings]




    return df1, df2

In [ ]:
browser = webdriver.Chrome()
browser.get('https://www.racingpost.com/profile/horse/772706/aiken/form')
time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))
    
time.sleep(2)

element2 = browser.find_elements_by_xpath("//td[@class='ui-table__cell']")
titles = [x.text for x in element2]
top_rpr = titles[10]
total_earnings = titles[7]
total_starts=titles[1]

age_element = browser.find_elements_by_xpath("//dt[@class='pp-definition__term']")
age_element_info = [x.text for x in age_element]
age=age_element_info[0]

sire = browser.find_elements_by_xpath("//a[@class='ui-link js-popupLink hp-horseDefinition__link']")
sire_info = [x.text for x in sire]

stud = browser.find_elements_by_xpath("//span[@class='hp-nameRow__studFee__value']")
stud_fee = 

In [ ]:
table_option

In [ ]:
comments = browser.find_elements_by_xpath("//dd[@class='pp-definition__description']")
sire_comments = [x.text for x in comments]
sire_comments[5]

In [ ]:
browser = webdriver.Chrome()
browser.get('https://www.racingpost.com/profile/horse/772706/aiken/form')
time.sleep(float(decimal.Decimal(random.randrange(200, 300))/100))
  

In [ ]:
sire_comments[5]

In [ ]:
comments

In [49]:
df4=pd.read_csv('progeny_stats_nov18.csv')